In [27]:
import numpy as np
import pandas as pd
from math import sqrt
from fbprophet import Prophet
from statsmodels.tsa.statespace.sarimax import SARIMAX
from tensorflow.keras.models import load_model
from datetime import datetime, date, timedelta

In [20]:
def prepare_data(path='/Users/andriikoval/Documents/uni/WS20-21/IML/ws2020-group-2/data/BTC-USD.csv', start_date='2020-01-01'):
    """Loads .csv file and prepares data for future use.

    Args:
        path (str): path to .csv file

    Returns:
        df: preprocessed DataFrame

    @author: Andrii Koval
    """
    df = pd.read_csv(path)
    df['Date'] = pd.to_datetime(df.Date, format='%Y-%m-%d')
    df['Diff'] = np.diff(df['Close'], prepend=[0])
    df['Direction'] = np.where(df['Diff'] >= 0, 'green', 'red')
    df['Diff_abs'] = np.abs(df['Diff'])
    df = df[df['Date'] > start_date]

    return df

In [35]:
df = prepare_data()
df = df.rename(columns={"Date": "ds", "Close": "y"})

model = Prophet()
model.fit(df)

today = datetime.today()
end = datetime.strptime('2021-03-01', '%Y-%m-%d')
diff = (end - today).days
future = model.make_future_dataframe(periods=diff, freq='D')
forecast = model.predict(future)

forecast['y_actual'] = model.history['y']
forecast = forecast.fillna(0)

forecast = forecast[forecast['ds'] < '2021-02-05']
forecast['error'] = pow((forecast['y_actual'] - forecast['yhat']), 2)
print('Root Mean Squared Error: ', sqrt(forecast['error'].mean()))

forecast.tail()

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
Root Mean Squared Error:  1786.1781454204304


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat,y_actual,error
391,2021-01-31,36637.389844,34276.763921,38684.556484,36637.389844,36637.389844,-85.083359,-85.083359,-85.083359,-85.083359,-85.083359,-85.083359,0.0,0.0,0.0,36552.306485,33114.359375,1.181948e+07
392,2021-02-01,36907.029938,34567.614021,39007.647668,36907.029938,36907.029938,-75.531140,-75.531140,-75.531140,-75.531140,-75.531140,-75.531140,0.0,0.0,0.0,36831.498798,33537.175781,1.085256e+07
393,2021-02-02,37176.670033,34805.171432,39363.384682,37176.670033,37176.670033,-50.130031,-50.130031,-50.130031,-50.130031,-50.130031,-50.130031,0.0,0.0,0.0,37126.540002,35510.289062,2.612267e+06
394,2021-02-03,37446.310127,35298.300523,39840.884409,37446.310127,37446.310127,71.859866,71.859866,71.859866,71.859866,71.859866,71.859866,0.0,0.0,0.0,37518.169993,37472.089844,2.123380e+03
395,2021-02-04,37715.950221,35328.097164,40111.786382,37715.950221,37715.950221,2.168784,2.168784,2.168784,2.168784,2.168784,2.168784,0.0,0.0,0.0,37718.119005,36926.066406,6.273473e+05
